# MobileNet - KHOTAA Diabetic Foot Ulcer Classification

## 1. Imports & Configuration

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
import numpy as np
from sklearn.model_selection import StratifiedKFold

sys.path.append('../')
sys.path.append('./')

from dataset_loader import SplitFolderDatasetLoader
from dataset_preprocessing import DFUPreprocessing
from utils.checkpoint_manager import CheckpointManager
from utils.training_engine import TrainingEngine, create_optimizer
from utils.metrics_evaluator import (
    calculate_metrics, print_metrics, plot_confusion_matrix,
    plot_roc_curve, plot_training_history
)

print("✓ Imports complete")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load Dataset

In [ ]:
# Load dataset
loader = SplitFolderDatasetLoader(root_dir='../../dfu-dataset-annotated-into-4-classes')
classes = loader.get_classes()
num_classes = loader.get_num_classes()

print(f"Classes: {classes}")
print(f"Number of classes: {num_classes}")

# Initialize preprocessing
preprocessor = DFUPreprocessing()
train_transform = preprocessor.get_train_transforms()
val_test_transform = preprocessor.get_valid_test_transforms()

# Dataset class
class DFUDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        from PIL import Image
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Prepare data for cross-validation
X_train, y_train = loader.load_split_paths('train', shuffle=True)
X_val, y_val = loader.load_split_paths('valid')
X_all = X_train + X_val
y_all = np.concatenate([y_train, y_val])

# Test set (untouched until final evaluation)
X_test, y_test = loader.load_split_paths('test')
test_dataset = DFUDataset(X_test, y_test, transform=val_test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize 5-fold stratified cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"\nTotal training samples (train+valid): {len(X_all)}")
print(f"Test samples: {len(X_test)}")
print("✓ Dataset loaded and ready for 5-fold cross-validation")

## 3. Model Definition

In [ ]:
# Setup device and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

print(f"Device: {device}")

# Create MobileNetV2 model
def create_mobilenet_model(num_classes=4, pretrained=True):
    """
    Create MobileNetV2 model for DFU classification.
    
    Args:
        num_classes: Number of output classes (4 for DFU grades)
        pretrained: Use ImageNet pretrained weights
    
    Returns:
        MobileNetV2 model configured for DFU classification
    """
    if pretrained:
        model = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    else:
        model = models.mobilenet_v2(weights=None)
    
    # Modify final classifier layer
    # MobileNet classifier is: Linear(1280 -> 1000)
    # Replace with: Linear(1280 -> num_classes)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    return model

# Test model creation
test_model = create_mobilenet_model(num_classes=num_classes)
print(f"\n✓ MobileNetV2 model created")
print(f"Input size: 224x224")
print(f"Output classes: {num_classes}")
print(f"Final classifier: {test_model.classifier[1]}")

## 4. Training

In [ ]:
# 5-Fold Cross-Validation Training
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_all, y_all), 1):
    print(f"\n{'='*60}\nFOLD {fold}/5\n{'='*60}")
    
    # Prepare fold data
    X_train_fold = [X_all[i] for i in train_idx]
    y_train_fold = y_all[train_idx]
    X_val_fold = [X_all[i] for i in val_idx]
    y_val_fold = y_all[val_idx]
    
    train_dataset = DFUDataset(X_train_fold, y_train_fold, transform=train_transform)
    val_dataset = DFUDataset(X_val_fold, y_val_fold, transform=val_test_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
    
    # Create model
    model = create_mobilenet_model(num_classes=num_classes, pretrained=True)
    model = model.to(device)
    
    # Setup optimizer using helper function (SGD with momentum=0.8)
    optimizer = create_optimizer(model, lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    checkpoint_manager = CheckpointManager(checkpoint_dir=f'checkpoints/mobilenet_fold{fold}')
    engine = TrainingEngine(model=model, device=device)
    
    # Train
    history = engine.train(
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=30,
        scheduler=scheduler,
        checkpoint_manager=checkpoint_manager,
        early_stopping_patience=7,
        use_early_stopping=True,
        verbose=True
    )
    
    # Store results
    best_val_acc = max(history['val_acc'])
    fold_results.append({
        'fold': fold,
        'best_val_acc': best_val_acc,
        'final_val_acc': history['val_acc'][-1],
        'stopped_epoch': history['stopped_epoch'],
        'history': history
    })
    print(f"Fold {fold} - Best Acc: {best_val_acc*100:.2f}% (stopped at epoch {history['stopped_epoch']})")

# Cross-validation summary
avg_acc = np.mean([r['best_val_acc'] for r in fold_results])
std_acc = np.std([r['best_val_acc'] for r in fold_results])
avg_epochs = np.mean([r['stopped_epoch'] for r in fold_results])

print(f"\n{'='*60}")
print(f"5-FOLD CROSS-VALIDATION RESULTS")
print(f"{'='*60}")
print(f"Mean Accuracy: {avg_acc*100:.2f}% ± {std_acc*100:.2f}%")
print(f"Average Epochs: {avg_epochs:.1f}")
print(f"\nIndividual Fold Results:")
for r in fold_results:
    print(f"  Fold {r['fold']}: {r['best_val_acc']*100:.2f}% (epoch {r['stopped_epoch']})")
print(f"{'='*60}")

## 5. Evaluation & Plots

In [ ]:
# Test Set Evaluation
print("\n" + "="*60)
print("TEST SET EVALUATION")
print("="*60)

# Load best fold model
best_fold_idx = np.argmax([r['best_val_acc'] for r in fold_results])
best_fold_num = fold_results[best_fold_idx]['fold']

print(f"Loading best model from Fold {best_fold_num}")

model = create_mobilenet_model(num_classes=num_classes, pretrained=True)
model = model.to(device)

checkpoint_manager = CheckpointManager(checkpoint_dir=f'checkpoints/mobilenet_fold{best_fold_num}')
checkpoint_manager.load_best_model(model, metric_name='accuracy')

engine = TrainingEngine(model=model, device=device)

# Evaluate with inference time tracking
test_loss, test_acc, predictions, true_labels, inference_time = engine.evaluate(
    test_loader, 
    criterion, 
    measure_inference_time=True
)

print(f"\nTest Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"\nInference Time Statistics:")
print(f"  Total Time: {inference_time['total_time']:.4f}s")
print(f"  Avg Time/Batch: {inference_time['avg_time_per_batch']*1000:.2f}ms ± {inference_time['std_time_per_batch']*1000:.2f}ms")
print(f"  Avg Time/Image: {inference_time['avg_time_per_image']*1000:.2f}ms")
print(f"  Throughput: {inference_time['images_per_second']:.1f} images/second")

# Get probabilities for AUC
model.eval()
all_probs = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.to(device))
        probs = torch.softmax(outputs, dim=1)
        all_probs.append(probs.cpu().numpy())

y_pred_proba = np.vstack(all_probs)

# Calculate all metrics
metrics = calculate_metrics(
    y_true=true_labels,
    y_pred=predictions,
    y_pred_proba=y_pred_proba,
    class_names=classes,
    average='macro'
)

print("\n" + "="*60)
print_metrics(metrics, title="MobileNetV2 Test Results")
print("="*60)

# Visualizations
import os
os.makedirs('results', exist_ok=True)

# Confusion Matrix
plot_confusion_matrix(
    y_true=true_labels,
    y_pred=predictions,
    class_names=classes,
    normalize=True,
    save_path='results/mobilenet_confusion_matrix.png'
)
print("\n✓ Confusion matrix saved to results/mobilenet_confusion_matrix.png")

# ROC Curve
plot_roc_curve(
    y_true=true_labels,
    y_pred_proba=y_pred_proba,
    class_names=classes,
    save_path='results/mobilenet_roc_curve.png'
)
print("✓ ROC curve saved to results/mobilenet_roc_curve.png")

# Training History (best fold)
plot_training_history(
    fold_results[best_fold_idx]['history'],
    save_path='results/mobilenet_training_history.png'
)
print("✓ Training history saved to results/mobilenet_training_history.png")

# Summary for Model Comparison
print("\n" + "="*60)
print("SUMMARY FOR MODEL COMPARISON")
print("="*60)
print(f"Model: MobileNetV2")
print(f"Cross-Validation Accuracy: {avg_acc*100:.2f}% ± {std_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test F1-Score: {metrics['f1_score']:.4f}")
print(f"Test MCC: {metrics['mcc']:.4f}")
print(f"Test AUC: {metrics['auc']:.4f}")
print(f"Average Training Epochs: {avg_epochs:.1f}")
print(f"Inference Time: {inference_time['avg_time_per_image']*1000:.2f}ms per image")
print(f"Throughput: {inference_time['images_per_second']:.1f} images/second")
print("="*60)

## 6. Save Results for Model Comparison

Save the results for later comparison with other models (ResNet50, ResNet101, DenseNet, GoogLeNet, EfficientNetV2S, PFCNN+DRNN).

In [ ]:
# Save results for model comparison
import json

mobilenet_results = {
    'model_name': 'MobileNetV2',
    'cv_results': {
        'val_accuracy': {'mean': float(avg_acc), 'std': float(std_acc)},
        'avg_epochs': float(avg_epochs),
        'fold_results': [
            {
                'fold': r['fold'],
                'best_val_acc': float(r['best_val_acc']),
                'stopped_epoch': int(r['stopped_epoch'])
            }
            for r in fold_results
        ]
    },
    'test_results': {
        'test_accuracy': float(test_acc),
        'test_loss': float(test_loss),
        'precision': float(metrics['precision']),
        'recall': float(metrics['recall']),
        'f1_score': float(metrics['f1_score']),
        'specificity': float(metrics['specificity']),
        'sensitivity': float(metrics['sensitivity']),
        'mcc': float(metrics['mcc']),
        'auc': float(metrics['auc'])
    },
    'inference_time': {
        'total_time': float(inference_time['total_time']),
        'avg_time_per_image_ms': float(inference_time['avg_time_per_image'] * 1000),
        'throughput_fps': float(inference_time['images_per_second'])
    }
}

# Save to JSON
os.makedirs('results', exist_ok=True)
with open('results/mobilenet_results.json', 'w') as f:
    json.dump(mobilenet_results, f, indent=4)

print("✓ Results saved to results/mobilenet_results.json")
print("\nThese results can be used with the ModelComparison utility:")
print("from utils.model_comparison import ModelComparison")
print("comparison = ModelComparison()")
print("comparison.add_model_result(**mobilenet_results)")
print("\n✓ MobileNetV2 training complete!")